# TEST_SENTIMENT – Phase exploratoire

Objectif :
Explorer les catégories émotionnelles présentes dans les tickets
à l’aide d’un modèle LLM.

Ce notebook correspond à une phase expérimentale ayant conduit
à la définition de la taxonomie finale stockée dans TAXONOMY_SENTIMENT.

Les résultats ont été analysés et consolidés manuellement
avant intégration dans le pipeline de production.


In [ ]:
WITH tickets_sample AS (
    SELECT body
    FROM projet_tal.public.data
    WHERE body IS NOT NULL
      AND LENGTH(body) > 50
    ORDER BY RANDOM()
    LIMIT 50
),
all_tickets AS (
    SELECT
        LISTAGG(body, '\n---\n') AS tickets_text
    FROM tickets_sample
)
SELECT
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-8b',
        CONCAT(
            'Voici des tickets de support utilisateur.\n',
            'Analyse l’ensemble des tickets ci-dessous et propose ',
            'une liste de 4 à 10 sentiments principaux.\n\n',
            'Contraintes :\n',
            '- Sentiments généraux (pas trop spécifiques)\n',
            '- Pas de synonymes proches\n',
            '- UN mot maximum pour nommer le sentiment\n',
            '- Réponse uniquement sous forme de liste\n\n',
            'Tickets:\n"""',
            tickets_text,
            '"""'
        )
    ) AS sentiments_proposes
FROM all_tickets;


In [ ]:
SHOW MODELS IN ACCOUNT;


In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';


In [ ]:
SELECT CURRENT_ROLE();



In [ ]:
SELECT
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-8b',
        'Donne 3 sentiments exprimés dans des tickets de support.'
    ) AS test;


In [ ]:
CREATE OR REPLACE TEMP TABLE ticket_batches AS
SELECT
    NTILE(50) OVER (ORDER BY RANDOM()) AS batch_id,
    body
FROM projet_tal.public.data;


In [ ]:
CREATE OR REPLACE TEMP TABLE batch_sentiments AS
SELECT
    batch_id,
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-8b',
        CONCAT(
            'Voici un lot de tickets de support.\n\n',
            'Identifie les 5 à 8 sentiments principaux exprimés ',
            'dans ces tickets.\n\n',
            'Contraintes :\n',
            '- Sentiments généraux\n',
            '- Pas de synonymes proches\n',
            '- UN mot maximum pour nommer le sentiment\n',
            '- Réponse sous forme de liste simple\n\n',
            'Tickets :\n"""',
            LISTAGG(body, '\n---\n'),
            '"""'
        )
    ) AS sentiments_list
FROM ticket_batches
GROUP BY batch_id;


In [ ]:
select *
from batch_sentiments

Frustration: 5,577 tickets

Concern: 4,266 tickets

Confusion: 2,939 tickets

Disappointment: 2,331 tickets

Uncertainty: 2,055 tickets

Urgency: 1,913 tickets

Curiosity: 1,322 tickets

Security: 1,090 tickets

Appreciation: 838 tickets

Integration: 665 tickets

Performance: 635 tickets

Anxiety: 474 tickets

Request: 311 tickets

Data Analysis: 279 tickets

Request for Information: 248 tickets

Disruption: 236 tickets

Technical Issues: 224 tickets

Support: 212 tickets

Hope: 203 tickets

Compliance: 179 tickets

a retenir : 
1. Frustration
2. Stress
3. Confusion
5. Insatisfaction
6. positif
6. Neutralité


In [ ]:
SELECT
    body,
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-8b',
        CONCAT(
            'Tu es un assistant chargé de détecter le ressenti émotionnel ',
            'd’un utilisateur à partir d’un ticket de support.\n\n',
            'Choisis UN SEUL sentiment dominant parmi la liste suivante :\n',
            'Frustration, Stress, Confusion, Insatisfaction, Positif, Neutralité.\n\n',
            'Règles :\n',
            '- Ne propose aucun autre sentiment\n',
            '- Réponds uniquement par le nom exact du sentiment\n\n',
            'Ticket utilisateur :\n"""',
            body,
            '"""'
        )
    ) AS sentiment
FROM projet_tal.public.data
WHERE subject like 'VPN Access Issue';
